### 80. ID番号への変換

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
train = pd.read_csv('data/chap6/train.txt',sep='\t',header=None)
valid = pd.read_csv('data/chap6/valid.txt',sep='\t',header=None)
test = pd.read_csv('data/chap6/test.txt',sep='\t',header=None)

In [ ]:
vectorizer = CountVectorizer(min_df=2)
train_title = train.iloc[:,1].str.lower()
cnt = vectorizer.fit_transform(train_title).toarray()
sm = cnt.sum(axis=0)
idx = np.argsort(sm)[::-1]
words = np.array(vectorizer.get_feature_names())[idx]

In [ ]:
train_title = train[1].str.lower()
cnt = vectorizer.fit_transform(train_title).toarray()
sm = cnt.sum(axis=0)
idx = np.argsort(sm)[::-1]
words = np.array(vectorizer.get_feature_names())[idx]

In [ ]:
d = dict()
for i in range(len(words)):
    d[words[i]] = i+1
def get_id(sentence):
      r = []
    for word in sentence:
        r.append(d.get(word,0))
        return r

def df2id(df):
    ids = []
    for i in df.iloc[:,1].str.lower():
        ids.append(get_id(i.split()))
    return ids

In [ ]:
X_train = df2id(train)
X_valid = df2id(valid)
X_test = df2id(test)

In [ ]:
!git add .

In [ ]:
!git commit -m "add chapter9"

In [ ]:
!git push

### 81. RNNによる予測

In [ ]:
import torch

In [ ]:
dw = 300
dh = 50
class RNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.emb = torch.nn.Embedding(len(words)+1,dw)
        self.rnn = torch.nn.RNN(dw,dh,batch_first=True)
        self.linear = torch.nn.Linear(dh,4)
        self.softmax = torch.nn.Softmax()
    def forward(self, x, h=None):
        x = self.emb(x)
        y, h = self.rnn(x, h)
        y = y[:,-1,:] # 最後のステップ
        y = self.linear(y)
        y = self.softmax(y)
        return y